## Goals for the next meeting (Fri, 10/2)

+ impute missing values using KNN
+ complete PCA for credit and medical payment
+ potentially fit the baseline model!!


Updated: Wed, 9/30

In [1]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD

## Import variable files

In [2]:
f = open('./health-PCA.txt', 'r')
health_pca_var_names = f.read().split("\n")
f = open('./credit-PCA.txt', 'r')
credit_pca_var_names = f.read().split("\n")
f = open('./medical-payment-PCA.txt', 'r')
med_pay_pca_var_names = f.read().split("\n")
f = open('./other-variables.txt', 'r')
var_names = f.read().split("\n")

In [3]:
health_pca_var_names = [var for var in pd.read_fwf('./health-PCA.txt',header=None)[0]]
credit_pca_var_names = [var for var in pd.read_fwf('./credit-PCA.txt',header=None)[0]]
med_pay_pca_var_names = [var for var in pd.read_fwf('./medical-payment-PCA.txt',header=None)[0]]
var_names = [var for var in pd.read_fwf('./other-variables.txt',header=None)[0]]

## Read datasets

Divided the original dataset into 4 smaller subsets.

+ `health_df`: dataset for health condition-related variables (no missing value - PCA is ok)
+ `credit_df`: dataset for financial information (need to impute missing values)
+ `med_pay_df`: dataset for medical payment related variables (need to impute missing values)
+  `df`: dataset for variables that do not need PCA (need to impute missing values)

In [4]:
health_df = pd.read_csv('./2020_Competition_Training.csv', usecols=health_pca_var_names,low_memory = False)
credit_df = pd.read_csv('./2020_Competition_Training.csv', usecols=credit_pca_var_names,low_memory = False)
med_pay_df = pd.read_csv('./2020_Competition_Training.csv', usecols=med_pay_pca_var_names,low_memory = False)
df = pd.read_csv('./2020_Competition_Training.csv', usecols=var_names,low_memory = False)

## PCA for `health_df`

In [5]:
pca_health = PCA(n_components = 80)
X = pca_health.fit_transform(health_df)

In [6]:
health_df.shape

(69572, 224)

In [7]:
sum(pca_health.explained_variance_ratio_)

0.8961624187712404

Reduced dimentions from 224 to 80 and retained ~ 90% variance

## The rest of the datasets

+ The remaining datasets contains NaN values -- PCA or other algorithms cannot be done. 
+ Need to impute missing values.
+ Current idea of how to impute: KNN

See this [reference](https://towardsdatascience.com/xgboost-is-not-black-magic-56ca013144b4) and [this](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) for implementation.

In [8]:
credit_df.shape

(69572, 20)

In [9]:
# All rows with missing values in credit_df
credit_missing = credit_df[credit_df.isnull().any(axis=1)]
credit_missing

,credit_bal_autobank,credit_bal_autofinance,credit_bal_bankcard_severederog,credit_bal_consumerfinance,credit_bal_heloc_60dpd,credit_bal_heloc_severederog,credit_bal_mtg_90to119dpd,credit_bal_mtg_bankruptcy,credit_bal_mtg_severederog,credit_bal_mtgcredit_new,credit_bal_studentloan_60dpd,credit_bal_totalallcredit_60dpd,credit_bal_totalallcredit_60to89dpd,credit_bal_totalallcredit_90to119dpd,credit_bal_totalallcredit_new,credit_highcrd_consumerfinance_new,credit_minmob_1stmtgcredit,credit_minmob_agencyfirstmtg,credit_minmob_mtgcredit,credit_minmob_nonagn
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
med_pay_df.shape

(69572, 13)

In [11]:
# NA in med_pay_df
med_pay_missing = med_pay_df[med_pay_df.isnull().any(axis=1)]
med_pay_missing

,cms_ma_risk_score_nbr,cms_partd_ra_factor_amt,cms_risk_adj_payment_rate_a_amt,cms_risk_adj_payment_rate_b_amt,cms_risk_adjustment_factor_a_amt,cms_rx_risk_score_nbr,cms_tot_ma_payment_amt,cms_tot_partd_payment_amt,rev_cms_clinic_ind,rev_cms_er_ind,rev_cms_icu_ind,rev_cms_lab_ind,rev_cms_phar_ind
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1,1
69520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
69542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1,1
69543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0


In [12]:
df[df.isnull().any(axis = 1)].shape

(21582, 40)

In [13]:
# NaN in df
df[df.isnull().any(axis = 1)]

,sex_cd,est_age,smoker_current_ind,smoker_former_ind,lang_spoken_cd,mabh_seg,cci_score,dcsi_score,fci_score,hcc_weighted_sum,...,phy_em_pe_ind,phy_em_pi_ind,phy_em_px_ind,prov_fb_ind,prov_pcp_ind,prov_sp_ind,rucc_category,rx_overall_pmpm_ct,total_outpatient_visit_ct_pmpm,total_physician_office_visit_ct_pmpm
0,F,62,1,0,ENG,UNK,3.0,1,2,0,...,0,0,1,1,1,1,1-Metro,0.084756,0.000000,0.250900
3,M,75,0,0,ENG,H6,3.0,0,2,0,...,0,0,1,0,1,1,7-Nonmetro,2.333333,0.083333,0.250000
8,F,56,0,0,ENG,H3,6.0,3,1,0,...,1,0,1,1,1,1,1-Metro,3.433116,0.250324,0.610793
11,F,67,0,0,SPA,UNK,6.0,1,4,0,...,0,0,1,1,1,1,1-Metro,12.909578,0.831499,2.838363
15,M,60,0,1,ENG,H2,4.0,2,2,3,...,0,0,1,0,1,1,2-Metro,2.299222,0.125749,0.581078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69564,F,70,0,0,ENG,UNK,4.0,3,5,4,...,0,0,1,1,1,1,2-Metro,6.876191,0.200000,0.912677
69565,F,18,0,0,ENG,UNK,0.0,0,0,0,...,0,0,0,0,0,0,6-Nonmetro,0.384005,0.000000,0.000000
69567,F,72,1,0,ENG,H7,5.0,2,2,2,...,0,0,1,1,1,1,1-Metro,3.916667,0.650000,1.333333
69568,M,75,0,0,ENG,C4,9.0,3,2,14,...,0,0,1,1,1,1,1-Metro,4.872273,1.106683,2.157150


In [14]:
df_missing=df.isnull().any()

In [15]:
df_missing[df_missing == True].index

Index(['cms_ra_factor_type_cd', 'cons_cmys', 'cons_hcaccprf_h',
       'cons_hcaccprf_p', 'cons_hhcomp', 'cons_homstat', 'cons_n2029_y',
       'cons_n2mob', 'cons_n2pbl', 'cons_n2pmv', 'cons_n65p_y',
       'cons_retail_buyer', 'hedis_dia_hba1c_ge9', 'hedis_dia_hba1c_test',
       'hedis_dia_ma_nephr'],
      dtype='object')

In [16]:
df[df.isnull().any(axis = 1)][df_missing[df_missing == True].index]

,cms_ra_factor_type_cd,cons_cmys,cons_hcaccprf_h,cons_hcaccprf_p,cons_hhcomp,cons_homstat,cons_n2029_y,cons_n2mob,cons_n2pbl,cons_n2pmv,cons_n65p_y,cons_retail_buyer,hedis_dia_hba1c_ge9,hedis_dia_hba1c_test,hedis_dia_ma_nephr
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,Y
3,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,Y
8,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,Y
11,CP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,Y
15,CP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69564,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,N
69565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69567,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,Y
69568,CF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Y,Y


In [17]:
from sklearn.impute import KNNImputer

In [18]:
credit_df.sample(10)

,credit_bal_autobank,credit_bal_autofinance,credit_bal_bankcard_severederog,credit_bal_consumerfinance,credit_bal_heloc_60dpd,credit_bal_heloc_severederog,credit_bal_mtg_90to119dpd,credit_bal_mtg_bankruptcy,credit_bal_mtg_severederog,credit_bal_mtgcredit_new,credit_bal_studentloan_60dpd,credit_bal_totalallcredit_60dpd,credit_bal_totalallcredit_60to89dpd,credit_bal_totalallcredit_90to119dpd,credit_bal_totalallcredit_new,credit_highcrd_consumerfinance_new,credit_minmob_1stmtgcredit,credit_minmob_agencyfirstmtg,credit_minmob_mtgcredit,credit_minmob_nonagn
44308,5472.927989,5130.862707,349.418943,819.436502,24.994429,39.723341,10.526386,345.614037,266.596157,9125.147291,566.924916,3553.992120,309.527401,20.282256,16293.161310,788.543666,28.583767,15.899942,27.855381,9.519988
3213,4317.461317,2517.959277,399.555783,528.374975,24.936778,2.866883,70.895779,76.215218,32.787334,8041.671225,1893.741768,6748.269785,333.674651,230.994901,11981.760715,518.794088,21.566299,9.460330,22.222170,13.051494
2878,5957.968747,6726.431914,511.416480,611.364441,80.984915,18.240953,489.259079,169.383899,101.712643,17596.173660,1364.812301,6915.536698,624.534056,426.526852,15484.806231,735.526885,27.527001,17.986238,29.980380,11.013042
30232,5982.209071,4503.288370,243.261162,649.971825,14.650237,6.879945,253.761982,478.204677,37.117296,7161.768654,2335.465042,5267.073285,653.434568,336.395352,16453.804485,799.227948,27.522663,18.302015,28.473216,6.216454
59381,6310.586918,4049.737153,317.651916,641.478406,148.876428,150.106468,96.407884,148.816484,233.757731,7754.004725,1037.202387,3553.237995,334.392158,277.389003,17280.406454,692.576935,27.676928,16.651276,34.117087,11.696752
16171,4100.569828,5659.898125,275.015570,673.715987,37.144860,2.289875,61.576647,168.798679,36.274208,7110.213798,1547.763639,3844.373003,301.167700,108.142175,14741.032147,651.770173,21.271417,15.688108,23.783457,5.445062
61367,3111.447710,4965.983516,211.675891,843.434061,6.189463,7.683590,184.223217,79.270946,40.064922,7691.393532,1494.407553,2245.540645,235.389656,377.219095,13409.882571,703.862531,22.731728,18.221687,25.938662,7.575449
11663,5286.393968,4381.416478,255.858632,747.295909,52.984190,38.946131,35.391060,115.346888,133.124628,6297.306763,863.692885,2063.585569,384.573444,116.015966,13052.184633,727.865405,25.198801,17.090898,33.680914,9.044074
23490,3356.511247,7960.167146,340.048215,833.227517,85.934544,5.874713,73.991865,292.328307,40.267374,7820.891540,1801.104678,4836.852168,534.783054,148.795501,18222.436941,752.674446,20.726127,13.121313,24.587756,7.783763
43821,3830.573113,3926.501377,278.205913,1116.602154,0.259484,0.000000,47.838432,64.142995,23.186035,2134.358211,2361.721976,4693.502661,483.353196,305.745587,8357.179424,770.682587,19.450878,12.219039,19.911181,5.342156


In [19]:
imputer = KNNImputer(n_neighbors=5)
df1 = pd.DataFrame(imputer.fit_transform(credit_missing))
df1

,0,1,2
0,404.451994,3407.307692,4.188668
1,404.451994,3407.307692,4.188668
2,404.451994,3407.307692,4.188668
3,404.451994,3407.307692,4.188668
4,404.451994,3407.307692,4.188668
...,...,...,...
226,404.451994,3407.307692,4.188668
227,404.451994,3407.307692,4.188668
228,404.451994,3407.307692,4.188668
229,404.451994,3407.307692,4.188668


In [20]:
imputer = KNNImputer(n_neighbors=5)
df2 = pd.DataFrame(imputer.fit_transform(med_pay_missing))
df2

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.1782,2.4998,347.742,430.622,1.1116,0.9634,705.892,104.002,0.0,0.0,0.0,0.0,0.0
1,1.1782,2.4998,347.742,430.622,1.1116,0.9634,705.892,104.002,0.0,0.0,0.0,0.0,0.0
2,1.1782,2.4998,347.742,430.622,1.1116,0.9634,705.892,104.002,0.0,0.0,0.0,0.0,0.0
3,1.1782,2.4998,347.742,430.622,1.1116,0.9634,705.892,104.002,0.0,0.0,0.0,0.0,0.0
4,2.3188,1.1312,503.842,747.566,2.2482,1.3442,1507.278,104.148,0.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3879,1.7338,1.3750,524.030,610.704,1.1908,0.8642,1299.200,74.754,0.0,1.0,0.0,1.0,1.0
3880,1.1782,2.4998,347.742,430.622,1.1116,0.9634,705.892,104.002,0.0,0.0,0.0,0.0,0.0
3881,1.7338,1.3750,524.030,610.704,1.1908,0.8642,1299.200,74.754,0.0,1.0,0.0,1.0,1.0
3882,0.9192,0.5258,367.798,496.956,1.1216,0.7062,882.444,96.060,0.0,0.0,0.0,1.0,0.0
